# Building a Data Lake using AWS Glue

1. [Introduction](#Introduction)
2. [Activity 1 : CSV to Parquet conversion](#Activity-1-:-CSV-to-Parquet-conversion)
3. [Activity 2 : Building a Star Schema in your Datalake](#Activity-2-:-Building-a-Star-Schema-in-your-Datalake)
3. [Activity 3 : Building an AWS Glue Workflow](#Activity-3-:-Building-an-AWS-Glue-Workflow)
4. [Wrap-up](#Wrap-up)

## Introduction

In this notebook, we will use AWS Glue to perform 2 activities:
    
- Convert a CSV Dataset to Parquet partitioned out by key fields.
- Build a Star (Denormalized) Schema from an OLTP 3NF (3rd Normal Form) Schema.
- Finally, deploy the piplline to create an AWS Glue Workflow.

Let's start by connecting to our our AWS Glue Dev Endpoint - a persistent AWS Glue Spark  Development environment.

In [1]:
spark.version

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1588867810053_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

u'2.4.3'

In [3]:
spark.sql("show databases").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|databaseName|
+------------+
|     default|
+------------+

In [5]:
spark.sql("show tables").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

## Activity 1 : CSV to Parquet conversion


The 1st dataset we will be using is the NYC Taxi Trips CSV dataset with 1.2B records. We will partition the data in the analytics tier by vendor name, year and month, catalog this data in the AWS Glue Data Catalog. This dataset has 5 vendors and 8 years of data.

### Crawl the Source Data

The 1st step is to run the AWS Glue Crawler on the raw dataset to create the table in the AWS Glue Catalog.

Create and Execute a AWS Glue Crawler on the source data in S3

- Navigate to the AWS Glue console at Services -> AWS Glue
- From the left-hand panel menu, navigate to Data Catalog -> Crawlers.
- Click on the button ‘Add Crawler’ to create a new AWS Glue Crawler.
- Fields to fill in:
    - Page: Add information about your crawler
        - Crawler name: **nyc_trips_csv_crawler**
    - Page: Add a data store
        - Choose a data store: S3
        - Include path: **s3://###s3_bucket###/data/nyc_trips_csv/**
    - Page: Choose an IAM role
       - IAM Role: Choose an existing IAM role **glue-labs\*GlueServiceRole**
    - Page: Configure the crawler's output
        - Database: Click on ‘Add database’ and enter database name as **nyc_trips**.
- Click on the button ‘Finish’ to create the crawler.
- Select the new Crawler and click on 'Run crawler' to run the Crawler.

Once the data is crawled, which should take about a minute, we can view the database and tables in the AWS Glue Catalog and query the tables as well:

### Transform the data to Parquet

Note that Spark SQL works great in the AWS Glue environment. Let's query the table created:

In [6]:
spark.sql("use nyc_trips").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

++
||
++
++

In [7]:
spark.sql("show tables").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------+-----------+
| database|    tableName|isTemporary|
+---------+-------------+-----------+
|nyc_trips|nyc_trips_csv|      false|
+---------+-------------+-----------+

In [8]:
df = spark.sql("select * from nyc_trips.nyc_trips_csv")
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- vendor_name: string (nullable = true)
 |-- trip_pickup_datetime: string (nullable = true)
 |-- trip_dropoff_datetime: string (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- are_amt: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amt: double (nullable = true)
 |-- tolls_amt: double (nullable = true)
 |-- total_amt: double (nullable = true)
 |-- year: long (nullable = true)
 |-- month: long (nullable = true)

Let's now write an AWS Glue Spark job to convert this csv data into a columnar(parquet) format.

In [9]:
## We will simulate the Glue job arguments 
import sys
sys.argv = ["CSV2Parquet","--JOB_NAME", "CSV2Parquet"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Let's start the code for the AWS Glue Job:

In [10]:
## Glue boilerplate code

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import boto3, json

args = getResolvedOptions(sys.argv, ['JOB_NAME'])
print (args['JOB_NAME']+" START...")
if 'sc' not in vars(): sc = SparkContext()
glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

## Glue boilerplate code

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CSV2Parquet START...

In [11]:
db_name='nyc_trips'
tbl_name='nyc_trips_csv'
output_dir='s3://###s3_bucket###/data/nyc_trips_parquet/'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We can easily instantiate an AWS Glue DynamicFrame from the AWS Glue Catalog table:

In [12]:
# Read the input data
dyf = glueContext.create_dynamic_frame.from_catalog(database = db_name, table_name = tbl_name)
dyf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- vendor_name: string
|-- trip_pickup_datetime: string
|-- trip_dropoff_datetime: string
|-- passenger_count: long
|-- trip_distance: double
|-- payment_type: string
|-- are_amt: double
|-- surcharge: double
|-- mta_tax: double
|-- tip_amt: double
|-- tolls_amt: double
|-- total_amt: double
|-- year: long
|-- month: long

As we are not doing any transformations here, we can write out the data out to our Amazon S3 bucket in Parquet right way:

In [13]:
# Write the data out in Parquet
glueContext.write_dynamic_frame.from_options(frame = dyf, connection_type = "s3", connection_options = {"path": output_dir, "partitionKeys": ['vendor_name', 'year', 'month']}, format = "parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
## Glue boilerplate code

job.commit()
print (args['JOB_NAME']+" END...")

## Glue boilerplate code

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CSV2Parquet END...

### Crawl the Transformed Data

Now that the output data is in Amazon S3, let's crawl this dataset in AWS Glue and query this data using Amazon Athena.

- Navigate to the AWS Glue console at Services -> AWS Glue
- From the left-hand panel menu, navigate to Data Catalog -> Crawlers.
- Click on the button ‘Add Crawler’ to create a new AWS Glue Crawler.
- Fields to fill in:
    - Page: Add information about your crawler
        - Crawler name: **nyc_trips_parquet_crawler**
    - Page: Add a data store
        - Choose a data store: S3
        - Include path: **s3://###s3_bucket###/data/nyc_trips_parquet/**
    - Page: Choose an IAM role
       - IAM Role: Choose an existing IAM role **glue-labs\*GlueServiceRole**
    - Page: Configure the crawler's output
        - Database: Select database as **nyc_trips**
- Click on the button ‘Finish’ to create the crawler.
- Select the new Crawler and click on Run crawler to run the Crawler.

Once the Crawler run has completed, navigate to the Amazon Athena console, Services -> Athena to run Amazon Athena queries on this dataset.

Note: You may need to output location for Amazon Athena by clicking on Settings -> Query result location in the Amazon Athena console and setting the value to : 

**s3://###s3_bucket###/athena-query-results/**

We can also query the data using Spark SQL:


In [15]:
spark.sql("show tables").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----------------+-----------+
| database|        tableName|isTemporary|
+---------+-----------------+-----------+
|nyc_trips|    nyc_trips_csv|      false|
|nyc_trips|nyc_trips_parquet|      false|
+---------+-----------------+-----------+

In [16]:
spark.sql("select count(*) from nyc_trips_parquet").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|25955596|
+--------+

## Activity 2 : Building a Star Schema in your Datalake

In this activity, we will denormalize an OLTP 3NF schema to Parquet. This activity demonstrates the using AWS Glue operations to perform powerful data transformations on input data:

![alt text](../resources/denormalize.png "Building a Star Schema")

### Crawl the Source Data

The 1st step is to run the AWS Crawler on the raw dataset to create the tables in the AWS Glue Catalog.

- Navigate to the AWS Glue console at Services -> AWS Glue
- From the left-hand panel menu, navigate to Data Catalog -> Crawlers.
- Click on the button ‘Add Crawler’ to create a new AWS Glue Crawler.
- Fields to fill in:
    - Page: Add information about your crawler
        - Crawler name: **salesdb_crawler**
    - Page: Add a data store
        - Choose a data store: S3
        - Include path: **s3://###s3_bucket###/data/salesdb/**
    - Page: Choose an IAM role
       - IAM Role: Choose an existing IAM role **glue-labs\*GlueServiceRole**
    - Page: Configure the crawler's output
        - Database:  Click on ‘Add database’ and enter database name as **salesdb**.
- Click on the button ‘Finish’ to create the crawler.
- Select the new Crawler and click on Run crawler to run the Crawler.



In [17]:
spark.sql("use salesdb").show()
spark.sql("show tables").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

++
||
++
++

+--------+------------------+-----------+
|database|         tableName|isTemporary|
+--------+------------------+-----------+
| salesdb|          customer|      false|
| salesdb|     customer_site|      false|
| salesdb|           product|      false|
| salesdb|  product_category|      false|
| salesdb|       sales_order|      false|
| salesdb|   sales_order_all|      false|
| salesdb|sales_order_detail|      false|
| salesdb|          supplier|      false|
+--------+------------------+-----------+

### Transform the dataset

Let's now denormalize the source tables where applicable and write out the data in Parquet format to the destination location:

In [18]:
db_name='salesdb'
table1='customer'
table2='customer_site'
output_dir='s3://###s3_bucket###/data/sales_analytics/customer_dim'
print (output_dir)

# Read the Source Tables
cust_dyf = glueContext.create_dynamic_frame.from_catalog(database = db_name, table_name = table1)
cust_site_dyf = glueContext.create_dynamic_frame.from_catalog(database = db_name, table_name = table2)

# Join the two Source Tables
customer_dim_dyf = Join.apply(cust_dyf,cust_site_dyf,
                       'cust_id', 'cust_id').drop_fields(['cust_id'])

# Write the denormalized CUSTOMER_DIM table in Parquet
glueContext.write_dynamic_frame.from_options(frame = customer_dim_dyf, connection_type = "s3", connection_options = {"path": output_dir}, format = "parquet")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://###s3_bucket###/data/sales_analytics/customer_dim

In [19]:
table1='product_category'
table2='product'
output_dir='s3://###s3_bucket###/data/sales_analytics/product_dim/'
print (output_dir)

# Read the Source Tables
table1_dyf = glueContext.create_dynamic_frame.from_catalog(database = db_name, table_name = table1)
table2_dyf = glueContext.create_dynamic_frame.from_catalog(database = db_name, table_name = table2)

#Join the Source Tables
product_dim_dyf = Join.apply(table1_dyf,table2_dyf,
                       'category_id', 'category_id').drop_fields(['category_id'])

# Write the denormalized CUSTOMER_DIM table in Parquet
glueContext.write_dynamic_frame.from_options(frame = product_dim_dyf, connection_type = "s3", connection_options = {"path": output_dir}, format = "parquet")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://###s3_bucket###/data/sales_analytics/product_dim/

In [20]:
table1='supplier'
output_dir='s3://###s3_bucket###/data/sales_analytics/supplier_dim/'
print (output_dir)

# Read the Source Tables
table1_dyf = glueContext.create_dynamic_frame.from_catalog(database = db_name, table_name = table1)


# Write the denormalized CUSTOMER_DIM table in Parquet
glueContext.write_dynamic_frame.from_options(frame = table1_dyf, connection_type = "s3", connection_options = {"path": output_dir}, format = "parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://###s3_bucket###/data/sales_analytics/supplier_dim/

In [21]:
table1='sales_order_detail'
table2='sales_order'
output_dir='s3://###s3_bucket###/data/sales_analytics/sales_order_fact/'
print (output_dir)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://###s3_bucket###/data/sales_analytics/sales_order_fact/

For the 'sales_order_fact' table, we will try a different approach - 

- We will convert the AWS Glue DynamicFrame to a Spark DataFrame
- Register the Spark Dataframe to a Spark Temporary View
- Use Spark SQL to build the write out the target dataset.

This demonstrates that AWS Glue DynamicFrames and Spark Dataframes are interchangeable and you can get the best of both worlds by using both the options where suitable.

In [22]:
# Read the Source Tables
table1_dyf = glueContext.create_dynamic_frame.from_catalog(database = db_name, table_name = table1)
table2_dyf = glueContext.create_dynamic_frame.from_catalog(database = db_name, table_name = table2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
table1_dyf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- LINE_ID: int
|-- LINE_NUMBER: int
|-- ORDER_ID: int
|-- PRODUCT_ID: int
|-- QUANTITY: int
|-- UNIT_PRICE: decimal
|-- DISCOUNT: decimal
|-- SUPPLY_COST: decimal
|-- TAX: decimal

In [24]:
table2_dyf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- ORDER_ID: int
|-- SITE_ID: double
|-- ORDER_DATE: timestamp
|-- SHIP_MODE: string

In [25]:
table1_dyf.toDF().createOrReplaceTempView("sales_order_v")
table2_dyf.toDF().createOrReplaceTempView("sales_order_detail_v")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Write the denormalized SALES_ORDER_FACT table
df=spark.sql("SELECT a.*, b.site_id, b.order_date,b.ship_mode \
FROM sales_order_detail_v b, sales_order_v a \
WHERE a.order_id=b.order_id")
df.printSchema()
print(df.count())
df.coalesce(1).write.mode("OVERWRITE").parquet("s3://###s3_bucket###/data/sales_analytics/sales_order_fact/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- LINE_ID: integer (nullable = true)
 |-- LINE_NUMBER: integer (nullable = true)
 |-- ORDER_ID: integer (nullable = true)
 |-- PRODUCT_ID: integer (nullable = true)
 |-- QUANTITY: integer (nullable = true)
 |-- UNIT_PRICE: decimal(38,10) (nullable = true)
 |-- DISCOUNT: decimal(38,10) (nullable = true)
 |-- SUPPLY_COST: decimal(38,10) (nullable = true)
 |-- TAX: decimal(38,10) (nullable = true)
 |-- site_id: double (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- ship_mode: string (nullable = true)

98000

Note that we used the power of Spark SQL for this transformation instead of AWS Glue DynamicFrame transforms. This dataset is small so we also coalesced the number of partitions in the Spark dataframe to 1 to ensure only 1 file gets written to our output location.

In [28]:
%%sh
aws s3 ls s3://###s3_bucket###/data/sales_analytics/sales_order_fact/

2020-05-07 19:25:22    2207233 part-00000-0f24406f-95ce-4c36-918f-67c5107858a1-c000.snappy.parquet


Now that the output data is in Amazon S3, let's crawl this dataset in AWS Glue and query this data using Amazon Athena.

### Crawl the Transformed Data

- Navigate to the Glue console at Services -> Glue
- From the left-hand panel menu, navigate to Data Catalog -> Crawlers.
- Click on the button ‘Add Crawler’ to create a new Glue Crawler.
- Fields to fill in:
    - Page: Add information about your crawler
        - Crawler name: **sales_analytics_crawler**
    - Page: Add a data store
        - Choose a data store: S3
        - Include path: **s3://###s3_bucket###/data/sales_analytics/**
    - Page: Choose an IAM role
        - IAM Role: Choose an existing IAM role **glue-labs\*GlueServiceRole**
    - Page: Configure the crawler's output
        - Database:  Click on ‘Add database’ and enter database name as **sales_analytics**.
- Click on the button ‘Finish’ to create the crawler.
- Select the new Crawler and click on Run crawler to run the Crawler.


In [29]:
spark.sql("use sales_analytics").show()
spark.sql("show tables").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

++
||
++
++

+---------------+--------------------+-----------+
|       database|           tableName|isTemporary|
+---------------+--------------------+-----------+
|sales_analytics|        customer_dim|      false|
|sales_analytics|         product_dim|      false|
|sales_analytics|    sales_order_fact|      false|
|sales_analytics|        supplier_dim|      false|
|               |sales_order_detail_v|       true|
|               |       sales_order_v|       true|
+---------------+--------------------+-----------+

## Activity 3 : Building an AWS Glue Workflow

An AWS Glue workflow is an orchestration used to visualize and manage the relationship and execution of multiple AWS Glue triggers, jobs and crawlers. Let's now build an AWS Glue Workflow for the same. 

The 1st step is to create the AWS Glue Jobs. As the AWS Glue ETL code is already staged in our Amazon S3 bucket, we will simply call the AWS Glue APIs to create the AWS Glue Jobs.

In [32]:
%local

import boto3

acct_number=boto3.client('sts').get_caller_identity().get('Account')
bucket='###s3_bucket###'

# Create the AWS Glue Spark Jobs
glue = boto3.client("glue")

for job_name in ['Load_SALES_ORDER_FACT', 'Load_PRODUCT_DIM', 'Load_CUSTOMER_DIM','Load_SUPPLIER_DIM']:
    response=glue.create_job(Name=job_name,
                         Role="arn:aws:iam::%s:role/###iam_role###"%acct_number,
                         ExecutionProperty={'MaxConcurrentRuns': 1},
                         Command={'Name': 'glueetl',
                                  'ScriptLocation': 's3://%s/scripts/%s.py'%(bucket,job_name),
                                  'PythonVersion': '3'},
                         DefaultArguments={'--TempDir': 's3://%s/temp'%bucket,
                                           '--enable-continuous-cloudwatch-log': 'true',
                                           '--enable-glue-datacatalog': '',
                                           '--enable-metrics': '',
                                           '--enable-spark-ui': 'true',
                                           '--spark-event-logs-path': 's3://%s/spark_glue_etl_logs/%s'%(bucket,job_name),
                                           '--job-bookmark-option': 'job-bookmark-disable',
                                           '--job-language': 'python',
                                           '--S3_BUCKET': bucket },
                         MaxRetries=0,
                         Timeout=2880,
                         MaxCapacity=3.0,
                         GlueVersion='1.0',
                         Tags={'Owner': 'Glue_Labs'}
                        )
    print (response)

{'Name': 'Load_SALES_ORDER_FACT', 'ResponseMetadata': {'RequestId': '1bfbe074-6b5b-4a9f-9995-f8b8245ef0fa', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 07 May 2020 19:30:17 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '32', 'connection': 'keep-alive', 'x-amzn-requestid': '1bfbe074-6b5b-4a9f-9995-f8b8245ef0fa'}, 'RetryAttempts': 0}}
{'Name': 'Load_PRODUCT_DIM', 'ResponseMetadata': {'RequestId': 'f700c51b-f42f-4a8c-9db1-be8ffddb6a5b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 07 May 2020 19:30:17 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '27', 'connection': 'keep-alive', 'x-amzn-requestid': 'f700c51b-f42f-4a8c-9db1-be8ffddb6a5b'}, 'RetryAttempts': 0}}
{'Name': 'Load_CUSTOMER_DIM', 'ResponseMetadata': {'RequestId': '0feeb12b-7fe4-4ee0-8e1c-43aee8289292', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 07 May 2020 19:30:17 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '28', 'connection': '

The Workflow consists of 3 AWS Glue triggers:

- The 1st OnDemand Trigger loads the Dimension tables.
- The 2nd Conditional Trigger loads the Fact table.
- The 3rd Conditional Trigger updated the table definitions in the Catalog.

In [33]:
%local

glue = boto3.client("glue")

# Create the AWS Glue Workflow
response = glue.create_workflow(
    Name='Sales_Analytics_Workflow',
    Description='Sales Analytics Workflow v1.0'
)
print (response)

# 1. The Trigger to load the Dimensions table
response = glue.create_trigger(
    Name='1_Load_Dimensions',
    WorkflowName='Sales_Analytics_Workflow',
    Type='ON_DEMAND',
    Actions=[{'JobName': 'Load_CUSTOMER_DIM',
    'Arguments': {'--job-bookmark-option': 'job-bookmark-disable'},
    'Timeout': 2880},
   {'JobName': 'Load_PRODUCT_DIM',
    'Arguments': {'--job-bookmark-option': 'job-bookmark-disable'},
    'Timeout': 2880},
   {'JobName': 'Load_SUPPLIER_DIM',
    'Arguments': {'--job-bookmark-option': 'job-bookmark-disable'},
    'Timeout': 2880}]
)
print (response)  

# 2. The Trigger to load the Facts table
response = glue.create_trigger(
    Name='2_Load_Facts',
    WorkflowName='Sales_Analytics_Workflow',
    Type='CONDITIONAL',
    #StartOnCreation=True,
    Actions=[{'JobName': 'Load_SALES_ORDER_FACT'}],
    Predicate= {'Logical': 'ANY',
    'Conditions': [{'LogicalOperator': 'EQUALS',
                  'JobName': 'Load_SUPPLIER_DIM',
                   'State': 'SUCCEEDED'},
                  {'LogicalOperator': 'EQUALS',
                   'JobName': 'Load_PRODUCT_DIM',
                   'State': 'SUCCEEDED'},
                  {'LogicalOperator': 'EQUALS',
                   'JobName': 'Load_CUSTOMER_DIM',
                   'State': 'SUCCEEDED'}]
               }
)
print (response)  

# Finally, the Trigger for the Crawler
response = glue.create_trigger(
    Name='3_Update_Catalog',
    WorkflowName='Sales_Analytics_Workflow',
    Type='CONDITIONAL',
    StartOnCreation=True,
    Actions=[{'CrawlerName': 'sales_analytics_crawler'}],
    Predicate= {'Logical': 'ANY',
   'Conditions': [{'LogicalOperator': 'EQUALS',
     'JobName': 'Load_SALES_ORDER_FACT',
     'State': 'SUCCEEDED'}]}
)
print (response)     

{'Name': 'Sales_Analytics_Workflow', 'ResponseMetadata': {'RequestId': 'a22ed418-687f-43c1-ad46-c7a56cad2d33', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 07 May 2020 19:31:19 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '35', 'connection': 'keep-alive', 'x-amzn-requestid': 'a22ed418-687f-43c1-ad46-c7a56cad2d33'}, 'RetryAttempts': 0}}
{'Name': '1_Load_Dimensions', 'ResponseMetadata': {'RequestId': '9c3aafb9-1bd9-4756-b660-01043a651516', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 07 May 2020 19:31:19 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '28', 'connection': 'keep-alive', 'x-amzn-requestid': '9c3aafb9-1bd9-4756-b660-01043a651516'}, 'RetryAttempts': 0}}
{'Name': '2_Load_Facts', 'ResponseMetadata': {'RequestId': '1d4692df-25f0-44ab-bac2-b23061ad4bf4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 07 May 2020 19:31:20 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '23', 'connection': 'k

Let's review the AWS Glue Workflow created:
    
- Navigate to the Glue Console at Service -> Glue
- From the left-hand panel menu, choose Workflows
- Select the Workflow 'Sales_Analytics_Workflow'.

Your workflow should look like this:

![title](../resources/Glue_Workflow.png)
  

Let us now run this workflow: 

- Select the workflow and click on 'Action - > Run' to launch the workflow
- You can view the run details and visually track the progress of each acitvity in the workflow from the 'History' tab by selecting the workflow run and clicking on 'View Run Details'

![title](../resources/View_Run_Details.png)


## Wrap-up

In this notebook, we ran exercises to perform: 

1. A CSV to Parquet conversion and observed how easy it is to transform and write data to an Amazon S3 bucket using AWS Glue, partitioned by key fields.
2. A more complex transformation - denormalizing of a 3NF OLTP schema, and we observed how easy it is to perform complex data transformations using the power of both AWS Glue DynamicFrames and Spark SQL.
3. We built and executed an AWS Glue Workflow to orchestrate multiple AWS Glue Jobs.
